# Source Extractor

In [ ]:
import sep
import numpy as np
import pandas as pd
import os
import shutil

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse

from astropy import wcs
from astropy.io import fits
from astropy.table import Table
from astroquery.gaia import Gaia
from astroquery.sdss import SDSS
from astroquery.simbad import Simbad

In [ ]:
# change file and directory
directory = "C:\\dir\\to\\file\\"
filen = "file.fits"

# choose whether or not to plot results (True will result in longer time!)
plotting = False

# open given FITS file
file = fits.open(directory+filen)
header = file[0].header
data = file[0].data

In [ ]:
# choose whether to apply mask to certain pixel values (True or False)
# this applys mask on inverted image, _bksub.fits image in DS9 to find a value to clip
mask = False

# choose mask values - check two graphs to make sure all annotations are red / no objects are red
mask_back_val = 20000
mask_obj_val = 4000

# choose to ignore pixels over or under given value ("over" or "under")
over_or_under = "under"



if mask:
    if over_or_under=="over":
        to_val = int(np.min(data)-10)
        data_bmasked = np.where(data>mask_back_val,to_val,data)
        data_masked = np.where(data>mask_obj_val,to_val,data)
        
        plt.figure(figsize=(40,20)).set_facecolor('lightgrey')
        plt.subplot(1,3,1)
        plt.title("BACKGROUND SUBTRACTED TRANSFORMED DATA")
        plt.imshow(data,cmap='gray',origin='lower',interpolation='none',vmin=np.min(data),vmax=np.max(data))
        plt.colorbar(fraction=0.046, pad=0.04)
        
        plt.subplot(1,3,2)
        plt.title("BACKGROUND MASKED IMAGE: ARE ALL ANNOTATIONS RED?")
        plt.imshow(data_bmasked,cmap='gray',interpolation='none',origin='lower',vmin=np.min(data),vmax=np.max(data))
        cmap = matplotlib.colormaps.get_cmap('gray').copy()
        cb = plt.colorbar(fraction=0.046, pad=0.04)
        cb.cmap.set_under('red')
        
        plt.subplot(1,3,3)
        plt.title("OBJECT MASKED IMAGE: ARE NO OBJECTS RED?")
        plt.imshow(data_masked,cmap='gray',interpolation='none',origin='lower',vmin=np.min(data),vmax=np.max(data))
        cmap = matplotlib.colormaps.get_cmap('gray').copy()
        cb = plt.colorbar(fraction=0.046, pad=0.04)
        cb.cmap.set_under('red')
        plt.show()
        
        data_bmask = data>mask_back_val
        data_mask = data>mask_obj_val
    elif over_or_under=="under":
        to_val = int(np.min(data)-10)
        data_bmasked = np.where(data<mask_back_val,to_val,data)
        data_masked = np.where(data<mask_obj_val,to_val,data)

        plt.figure(figsize=(40,20)).set_facecolor('lightgrey')
        plt.subplot(1,3,1)
        plt.title("BACKGROUND SUBTRACTED TRANSFORMED DATA")
        plt.imshow(data,cmap='gray',origin='lower',interpolation='none',vmin=np.min(data)+1,vmax=np.max(data))
        plt.colorbar(fraction=0.046, pad=0.04)
        
        plt.subplot(1,3,2)
        plt.title("BACKGROUND MASKED IMAGE: ARE ALL ANNOTATIONS RED?")
        plt.imshow(data_bmasked,cmap='gray',interpolation='none',origin='lower',vmin=np.min(data)+1,vmax=np.max(data))
        cmap = matplotlib.colormaps.get_cmap('gray').copy()
        cb = plt.colorbar(fraction=0.046, pad=0.04)
        cb.cmap.set_under('red')        
        
        plt.subplot(1,3,3)
        plt.title("OBJECT MASKED IMAGE: ARE NO OBJECTS RED?")
        plt.imshow(data_masked,cmap='gray',interpolation='none',origin='lower',vmin=np.min(data),vmax=np.max(data))
        cmap = matplotlib.colormaps.get_cmap('gray').copy()
        cb = plt.colorbar(fraction=0.046, pad=0.04)
        cb.cmap.set_under('red')
        plt.show()
        
        data_bmask = data<mask_back_val
        data_mask = data<mask_obj_val
    else:
        print("mask not applied: over_or_under variable spelled incorrectly")
else:
    data_bmask = data!=data
    data_mask = data!=data

In [ ]:
# simple "inversion" of the data
invdata = -data.astype(np.float64)

# find and subtract background, save file
bkg = sep.Background(invdata,mask=data_bmask,bw=64,bh=64,fw=3,fh=3)
sdata = invdata-bkg
fits.writeto(directory+filen[:-5]+'_bksub.fits',sdata,header,overwrite=True)

if plotting:
    # plot the above images
    plt.figure(figsize=(40,10)).set_facecolor('lightgrey')
    plt.subplot(1,4,1)
    plt.title("ORIGINAL DATA")
    plt.imshow(data,cmap='gray',origin="lower",interpolation='none')
    plt.colorbar(fraction=0.046, pad=0.04)
    plt.subplot(1,4,2)
    plt.title("TRANSFORMED DATA")
    plt.imshow(invdata,cmap='gray',origin="lower",interpolation='none')
    plt.colorbar(fraction=0.046, pad=0.04)
    plt.subplot(1,4,3)
    plt.title("BACKGROUND")
    plt.imshow(bkg,cmap='gray',origin="lower",interpolation='none')
    plt.colorbar(fraction=0.046, pad=0.04)
    plt.subplot(1,4,4)
    plt.title("BACKGROUND SUBTRACTED TRANSFORMED DATA")
    plt.imshow(sdata,cmap='gray',origin="lower",interpolation='none')
    plt.colorbar(fraction=0.046, pad=0.04)
    plt.show()

In [ ]:
# find objects in background subtracted transformed image using simple parameters
objects = sep.extract(sdata,5,err=bkg.globalrms,mask=data_mask,clean=False,minarea=25)
df = pd.DataFrame(objects)

if plotting:
    # BELOW GRAPHING CODE BASED: https://sep.readthedocs.io/en/v1.1.x/tutorial.html

    # plot background-subtracted image
    plt.figure(figsize=(40,40)).set_facecolor('lightgrey')
    ax = plt.gca()
    # not used: m, s = np.mean(sdata), np.std(sdata)
    plt.imshow(np.where(data_mask,int(np.min(sdata)-10),sdata),cmap='gray',origin='lower',interpolation='none',vmin=np.min(sdata),vmax=np.max(sdata))
    cmap = matplotlib.colormaps.get_cmap('gray').copy()
    cb = plt.colorbar(fraction=0.046, pad=0.04)
    cb.cmap.set_under('red')
    
    # plot an ellipse for each object
    for i in range(len(objects)):
        e = Ellipse(xy=(objects['x'][i], objects['y'][i]),
                    width=6*objects['a'][i],
                    height=6*objects['b'][i],
                    angle=objects['theta'][i] * 180. / np.pi)
        e.set_facecolor('none')
        e.set_edgecolor('orange')
        ax.add_artist(e)

    plt.title("OBJECTS")
    plt.show()
    
print(f'a total of {len(objects)} objects found')

In [ ]:
# convert source center xs and ys to ras and decs using FITS header
w = wcs.WCS(header)
ras,decs = w.all_pix2world(df["x"],df["y"],0)
df["ra"],df["dec"] = ras,decs

# calculate a circle that encompasses the entire plate's area
cra,cdec = w.all_pix2world(header["NAXIS1"]/2,header["NAXIS2"]/2,0)
radius = (((w.all_pix2world(header["NAXIS1"],header["NAXIS2"],0)[0]-w.all_pix2world(1,1,0)[0])**2+
          (w.all_pix2world(header["NAXIS1"],header["NAXIS2"],0)[1]-w.all_pix2world(1,1,0)[1])**2)**0.5)/2

# calculate a rectangle that encompasses the entire plate's area
ra1,dec1 = w.all_pix2world(1,1,0)
ra2,dec2 = w.all_pix2world(1,header["NAXIS2"],0)
ra3,dec3 = w.all_pix2world(header["NAXIS1"],1,0)
ra4,dec4 = w.all_pix2world(header["NAXIS1"],header["NAXIS2"],0)

In [ ]:
# save the objects derived from source extractor
df.to_csv(directory+filen[:-5]+'_obj.csv')
print("In directory: "+directory)
print("Background Subtracted Transformed FITS image saved in "+filen[:-5]+'_bksub.fits')
print("Source Extractor Objects saved in "+filen[:-5]+'_obj.csv')

print("\nSource Extraction Done!\nuse cells below to get catalog data")

---

# Catalogs

### GAIA catalog

In [ ]:
# run GAIA query in plate area (circle)
job = Gaia.launch_job_async(
    """
    SELECT 
        source_id,ra as ra_J2016,array_element(a0,1) as ra_J{0},dec as dec_J2016,array_element(a0,2) as dec_J{0},
        parallax as parallax_J2016,array_element(a0,3) as parallax_J{0},parallax_error,
        pmra as pmra_J2016,array_element(a0,4) as pmra_J{0},pmra_error,
        pmdec as pmdec_J2016,array_element(a0,5) as pmdec_J{0},pmdec_error,
        radial_velocity as radial_velocity_J2016,array_element(a0,6) as radial_velocity_J{0},
        phot_g_mean_mag,phot_bp_mean_mag,phot_bp_mean_flux_over_error,phot_rp_mean_mag,phot_rp_mean_flux_over_error,
        bp_rp,phot_variable_flag,classprob_dsc_combmod_galaxy
    FROM
       (
        SELECT TOP 500000 
            source_id,ra,dec,parallax,parallax_error,pmra,pmra_error,pmdec,pmdec_error,radial_velocity,
            phot_g_mean_mag,phot_bp_mean_mag,phot_bp_mean_flux_over_error,phot_rp_mean_mag,phot_rp_mean_flux_over_error,
            bp_rp,phot_variable_flag,classprob_dsc_combmod_galaxy,
            epoch_prop(ra,dec,parallax,pmra,pmdec,radial_velocity,2016.0,{0}) as a0
        FROM gaiadr3.gaia_source
        WHERE CONTAINS(POINT('ICRS',ra,dec),CIRCLE('ICRS',{1},{2},{3}))=1  AND  ((phot_bp_mean_mag + 0.9*bp_rp) <= 20.)
        ) as p
    """.format(2000,cra,cdec,radius),
    dump_to_file=True, output_format='csv')

# save and rename the output file in the correct place
os.replace(job.outputFile,filen[:-5]+'_gaia.csv')
shutil.move(filen[:-5]+'_gaia.csv',directory+filen[:-5]+'_gaia.csv')
gaia_data = pd.read_csv(directory+filen[:-5]+'_gaia.csv')
print(str(len(gaia_data))+" objects detected in field")
print("\nGaia Objects saved as "+filen[:-5]+'_gaia.csv')

### SDSS Catalog

In [ ]:
# star or galaxy
star_or_galaxy = "star"

# run SDSS query in plate area (rectangle)
query = """
        select top 500000
            objID, ra, dec, u, err_u, g, err_g, i, err_i, clean
        from {0}
        where (ra between {1} and {2}) and (dec between {3} and {4}) and g < 20 and clean=1.
        """.format(star_or_galaxy,np.min([ra1,ra2,ra3,ra4]),np.max([ra1,ra2,ra3,ra4]),np.min([dec1,dec2,dec3,dec4]),np.max([dec1,dec2,dec3,dec4]))
res = SDSS.query_sql(query,data_release=18)

# save output file
print(str(len(res))+" "+star_or_galaxy+"s detected in field")
res.write(directory+filen[:-5]+"_sdss-"+star_or_galaxy+".csv",overwrite=True)
print("\nSDSS Objects saved as "+filen[:-5]+"_sdss-"+star_or_galaxy+".csv")

### SIMBAD Catalog

In [ ]:
# run SIMBAD query in plate area (circle)
cSimbad = Simbad()
cSimbad.TIMEOUT = 1200
cSimbad.ROW_LIMIT = -1
cSimbad.add_votable_fields('ra(d)','dec(d)','otype','flux(u)','flux(g)','parallax','z_value','sp','pm','v*')
query = "region(circle, {0} {3}{1}, {2}d) & gmag < 19.5".format(cra,cdec,radius,"-" if cdec<0 else "+")
result_table = cSimbad.query_criteria(query)

# save output file
print(str(len(result_table))+" objects detected in field")
result_table.write(directory+filen[:-5]+"_simbad.csv",overwrite=True)
print("\nSIMBAD Objects saved as "+filen[:-5]+"_simbad.csv")

---